In [36]:
#1.0 안전신문고-전기차 충전방해 신고 통계 (서울 자치구별)

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

driver = webdriver.Chrome(service=Service("../driver/chromedriver"))
driver.set_window_size(1200, 1000)
driver_path = "https://www.safetyreport.go.kr/#introduction/safeSingoStatistics"
driver.get(driver_path)

In [1]:
driver.close()

NameError: name 'driver' is not defined

In [37]:
#1.1 기간별 
year_link = driver.find_element(By.XPATH, '''//*[@id="seachDateType"]/option[2]''')
year_link.click()

In [38]:
#1.2 지역 리스트 만들기
sido = driver.find_element(By.ID, "API_CTRD_CODE")
sido_tmp = sido.find_elements(By.TAG_NAME, "option")


sido_list = [sido_list_tmp.text for sido_list_tmp in sido_tmp]
sido_list = sido_list[1:]
sido_list

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '경기도',
 '강원특별자치도',
 '충청북도',
 '충청남도',
 '전북특별자치도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도',
 '세종특별자치시']

In [39]:
#1.3 콤보박스 서울특별시 선택
print(sido_list[0])
sido.send_keys(sido_list[0])

서울특별시


In [40]:
#1.4 서울특별시 월별 통계
search_box1 = driver.find_element(By.CSS_SELECTOR, "#C_FRM_DATE")
search_box1.clear()
search_box1.send_keys('2022-10-07')


search_box2 = driver.find_element(By.ID, "C_TO_DATE")
search_box2.clear()
search_box2.send_keys('2023-02-28')


search_link = driver.find_element(By.XPATH, '''//*[@id="form"]/fieldset/div/button''')
search_link.click()




In [ ]:
env_tmp = driver.find_element(By.ID, "grid")
env = env_tmp.find_element(By.XPATH, '''//*[@id="grid"]/tbody/tr[23]/td[2]/a''').text
env


In [41]:
#2.2 구 리스트 만들기

driver.implicitly_wait(3) # 이거하면 run all 가능

from selenium.webdriver.common.by import By

gu = driver.find_element(By.ID, "API_SIGNGU_CODE")
gu_list = gu.find_elements(By.TAG_NAME, "option")

gu_names = [gu_name.get_attribute("innerHTML") for gu_name in gu_list]
print(len(gu_names), gu_names)

gu_names.remove('선택')
print(len(gu_names), gu_names)


26 ['선택', '종로구', '중구', '용산구', '성동구', '동대문구', '중랑구', '광진구', '성북구', '강북구', '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구', '영등포구', '동작구', '관악구', '서초구', '강남구', '강동구', '송파구']
25 ['종로구', '중구', '용산구', '성동구', '동대문구', '중랑구', '광진구', '성북구', '강북구', '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구', '영등포구', '동작구', '관악구', '서초구', '강남구', '강동구', '송파구']


In [ ]:
gu.send_keys(gu_names[0])

In [45]:
#2-4. 구별로 이동 (25개 순차적으로 클릭) - 구 이동하려면 이걸 사용.

import time
from tqdm.notebook import tqdm

from selenium.webdriver.common.by import By # Selenium 웹 드라이버에서 사용되는 요소를 식별하는 데 필요한 By 클래스 가져오기

env_list = []

# 각 "구"로 이동
for gu_name in tqdm(gu_names):
    gu = driver.find_element(By.ID, "API_SIGNGU_CODE")
    gu.send_keys(gu_name)

    search_link = driver.find_element(By.XPATH, '''//*[@id="form"]/fieldset/div/button''')
    search_link.click()

    #민원건수
    env_tmp = driver.find_element(By.ID, "grid")
    env = env_tmp.find_element(By.XPATH, '''//*[@id="grid"]/tbody/tr[23]/td[2]/a''').text
    env_list.append(env)
    time.sleep(10)

  0%|          | 0/25 [00:00<?, ?it/s]

In [46]:
print(len(env_list),env_list)

25 ['988', '261', '324', '561', '482', '332', '307', '179', '479', '106', '213', '448', '233', '259', '477', '340', '513', '529', '193', '597', '348', '229', '599', '988', '476']


In [57]:
#2.6 Pandas

import pandas as pd

gu_env = {'구별' : gu_names, 
        '충전방해건수' : env_list       
        }
gu_env = pd.DataFrame(gu_env)
gu_env

,구별,충전방해건수
0,종로구,988
1,중구,261
2,용산구,324
3,성동구,561
4,동대문구,482
5,중랑구,332
6,광진구,307
7,성북구,179
8,강북구,479
9,도봉구,106


In [62]:
gu_env = gu_env.sort_values(by='구별',ascending=[True])


In [64]:
gu_env.reset_index(drop=True)

,구별,충전방해건수
0,강남구,599
1,강동구,988
2,강북구,479
3,강서구,340
4,관악구,348
5,광진구,307
6,구로구,513
7,금천구,529
8,노원구,213
9,도봉구,106


In [63]:
print(gu_env)

      구별 충전방해건수
22   강남구    599
23   강동구    988
8    강북구    479
15   강서구    340
20   관악구    348
6    광진구    307
16   구로구    513
17   금천구    529
10   노원구    213
9    도봉구    106
4   동대문구    482
19   동작구    597
13   마포구    259
12  서대문구    233
21   서초구    229
3    성동구    561
7    성북구    179
24   송파구    476
14   양천구    477
18  영등포구    193
2    용산구    324
11   은평구    448
0    종로구    988
1     중구    261
5    중랑구    332


In [66]:
gu_env.to_csv("../data/year_car_enroll.csv", sep=",", encoding="utf-8")